# Non-Intrusive Load Monitoring Framework

# Data

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%cd drive/My Drive/Saya

/content/drive/My Drive/Saya


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('ggplot')

In [0]:
ts = pd.read_excel('data/endtoend.xlsx', header=None, sep=',')

In [0]:
ts['timestamp'] = ts[0]
tmpstmp = np.array(ts.timestamp)
ts['pressure'] = ts[2]

In [6]:
ts.head()

,0,1,2,3,4,5,6,7,8,9,timestamp,pressure
0,CreatedDate,SupplyWaterTemperature,SupplyWaterPressure,BackWaterTemperature,BackWaterPressure,ValveStatus,TotalFlowTotalFlow,Flow,FlowRate,SendInterval,CreatedDate,SupplyWaterPressure
1,2018-06-09 10:44:00,66.56,80.041,0,0,1,94178.9,23544.7,0,60,2018-06-09 10:44:00,80.041
2,2018-06-09 10:45:00,66.605,80.041,0,0,1,94178.9,0,0,60,2018-06-09 10:45:00,80.041
3,2018-06-09 10:46:00,66.614,80.049,0,0,1,94178.9,0,0,60,2018-06-09 10:46:00,80.049
4,2018-06-09 10:47:00,66.47,80.04,0,0,1,94178.9,0,0,60,2018-06-09 10:47:00,80.04


In [0]:
ts.drop(ts.columns[:10], axis=1,inplace=True)

In [8]:
ts.head()

,timestamp,pressure
0,CreatedDate,SupplyWaterPressure
1,2018-06-09 10:44:00,80.041
2,2018-06-09 10:45:00,80.041
3,2018-06-09 10:46:00,80.049
4,2018-06-09 10:47:00,80.04


In [0]:
ts = ts.drop(ts.index[0])

In [10]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382249 entries, 1 to 382249
Data columns (total 2 columns):
timestamp    382249 non-null object
pressure     382249 non-null object
dtypes: object(2)
memory usage: 8.7+ MB


In [0]:
ts['timestamp'] = pd.to_datetime(ts['timestamp'])

In [24]:
ts.set_index(keys='timestamp', inplace=True)
ts = ts.tz_localize('UTC').tz_convert('US/Pacific')
ts = ts.ix[::60] #downsample dataset
ts.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,pressure
timestamp,
2019-02-20 00:00:00-08:00,100.004
2019-02-20 01:00:00-08:00,100.005
2019-02-20 02:00:00-08:00,100.005
2019-02-20 03:00:00-08:00,100.014
2019-02-20 04:00:00-08:00,100.004
2019-02-20 05:00:00-08:00,90.089
2019-02-20 06:02:00-08:00,90.082
2019-02-20 07:02:00-08:00,70.051
2019-02-20 08:02:00-08:00,90.057


# EDA

In [11]:
import datetime
ts.plot(figsize=(20,16), color='r')
plt.xlabel('Time (s)')
plt.ylabel('Water Consumed (Gallons)')
plt.title('Water Consumed vs. Time')

ValueError: ignored

ValueError: ignored

<Figure size 1440x1152 with 1 Axes>

# Time Series segmentation

In [25]:
ts['pressure'] = ts['pressure'].round(1)

AttributeError: ignored

In [26]:
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

matrix = []

for ts_i in window(ts.pressure, 30):
    matrix.append(ts_i)

matrix = np.array(matrix)
matrix.shape

(941, 30)

In [27]:
matrix

array([[100.004, 100.005, 100.005, ..., 100.005,  90.089,  90.088],
       [100.005, 100.005, 100.014, ...,  90.089,  90.088,  90.063],
       [100.005, 100.014, 100.004, ...,  90.088,  90.063,  90.05 ],
       ...,
       [ 90.089,  90.073,  90.069, ...,  90.062,  90.07 ,  90.064],
       [ 90.073,  90.069,  90.073, ...,  90.07 ,  90.064,  80.03 ],
       [ 90.069,  90.073,  90.089, ...,  90.064,  80.03 ,  90.067]])

In [28]:
dates_matrix = []

for time in window(ts.index, 30):
    dates_matrix.append(time)
    
dates_matrix = np.array(dates_matrix)
dates_matrix.shape

(941, 30)

# Make Labels for Segmented Time Series Data According to Given Information




*   Shower (Label 1) - This is the smallest amplitude repeating pulse at ??

*   Toilet (Label 2) - Runs for a duration of about ?? Starts at the same time everyday.

*   Faucet 1 (Label 3) - The most common repeating pulse. (At about ?? amplitude and a width of about ??minutes)
*   
Faucet 2 (Label 4) - Another repeating pulse but less frequent  (At about ?? amplitude and a width of about ??minutes)





In [0]:
labels = []
for ts_i in matrix:
    if any(ts_i <= 70): #Shower
        label = 1
    elif any((ts_i > 70) & (ts_i <= 80)): #Toilets
        label = 2
    elif any((ts_i > 80) & (ts_i < 90)): #Faucet 1
        label = 3
    elif any(ts_i >= 90): #Faucet 2
        label = 4
    else: #Other Fixtures (Unknown)
        label = 5
    labels.append(label)
labels = np.array(labels)



In [34]:

from collections import Counter
d = Counter(labels)
print (d)

Counter({2: 435, 1: 217, 4: 187, 3: 102})


# Supervised Learning: Time Series K-Nearest Neighbor Classification

In [0]:
from sklearn.metrics import classification_report
import numpy as np

class ts_KnnClassifier(object):
    def __init__(self):
        '''
        preds is a list of predictions that will be made.
        plotter indicates whether to plot each nearest neighbor as it is found.
        '''
        self.preds=[]

    def predict(self, X_train, X_test, y_train, w):
        '''
        1-nearest neighbor classification algorithm using LB_Keogh lower 
        bound as similarity measure. Option to use DTW distance instead
        but is much slower.
        '''
        for i in X_test:
            min_dist = float('inf')
            closest_seq_ind = []

            for ind, j in enumerate(X_train):
                if self.LB_Keogh(i, j, 5) < min_dist:
                    dist = self.DTWDistance(i, j, w)
                    if dist < min_dist:
                        min_dist = dist
                        closest_seq_ind = j
            self.preds.append(y_train[closest_seq_ind])

    def performance(self, true_results):
        '''
        If the actual test set labels are known, can determine classification
        accuracy.
        '''
        return classification_report(true_results, self.preds)

    def get_preds(self):
        return self.preds

    def DTWDistance(self, s1, s2, w=None):
        '''
        Calculates dynamic time warping Euclidean distance between two
        sequences. Option to enforce locality constraint for window w.
        '''
        DTW={}

        if w:
            w = max(w, abs(len(s1)-len(s2)))

            for i in range(-1,len(s1)):
                for j in range(-1,len(s2)):
                    DTW[(i, j)] = float('inf')

        else:
            for i in range(len(s1)):
                DTW[(i, -1)] = float('inf')
            for i in range(len(s2)):
                DTW[(-1, i)] = float('inf')

        DTW[(-1, -1)] = 0

        for i in range(len(s1)):
            if w:
                for j in range(max(0, i-w), min(len(s2), i+w)):
                    dist= (s1[i] - s2[j])**2
                    DTW[(i, j)] = dist + min(DTW[(i-1, j)], DTW[(i, j-1)], DTW[(i-1, j-1)])
            else:
                for j in range(len(s2)):
                    dist= (s1[i] - s2[j])**2
                    DTW[(i, j)] = dist + min(DTW[(i-1, j)], DTW[(i, j-1)], DTW[(i-1, j-1)])

        return np.sqrt(DTW[len(s1)-1, len(s2)-1])

    def LB_Keogh(self,s1,s2,r):
        '''
        Calculates LB_Keough lower bound to dynamic time warping. Linear
        complexity compared to quadratic complexity of dtw.
        '''
        LB_sum=0
        for ind, i in enumerate(s1):

            lower_bound=min(s2[(ind-r if ind-r >= 0 else 0):(ind + r)])
            upper_bound=max(s2[(ind-r if ind-r >= 0 else 0):(ind + r)])

            if i > upper_bound:
                LB_sum = LB_sum + (i - upper_bound)**2
            elif i < lower_bound:
                LB_sum = LB_sum + (i - lower_bound)**2

        return np.sqrt(LB_sum)

In [0]:

from sklearn.metrics import f1_score, precision_score, recall_score

y_predsCV = []

def performTimeSeriesCV(X_train, y_train, number_folds):
    """
    Given X_train and y_train (the test set is excluded from the Cross Validation),
    number of folds, the ML algorithm to implement and the parameters to test,
    the function acts based on the following logic: it splits X_train and y_train in a
    number of folds equal to number_folds. Then train on one fold and tests accuracy
    on the consecutive as follows:
    - Train on fold 1, test on 2
    - Train on fold 1-2, test on 3
    - Train on fold 1-2-3, test on 4
    ....
    Returns mean of test accuracies.
    """
    global y_predsCV
 
    print ('Size train set: ', X_train.shape)
    
    # k is the size of each fold. It is computed dividing the number of 
    # rows in X_train by number_folds. This number is floored and coerced to int
    k = int(np.floor(float(X_train.shape[0]) / number_folds))
    print ('Size of each fold: ', k)
    
    # initialize to zero the accuracies array. It is important to stress that
    # in the CV of Time Series if I have n folds I test n-1 folds as the first
    # one is always needed to train
    f1_scores = np.zeros(number_folds-1)
    precision_scores = np.zeros(number_folds-1)
    recall_scores = np.zeros(number_folds-1)
    
 
    # loop from the first 2 folds to the total number of folds    
    for i in range(2, number_folds + 1):
        print ('')
        
        # the split is the percentage at which to split the folds into train
        # and test. For example when i = 2 we are taking the first 2 folds out 
        # of the total available. In this specific case we have to split the
        # two of them in half (train on the first, test on the second), 
        # so split = 1/2 = 0.5 = 50%. When i = 3 we are taking the first 3 folds 
        # out of the total available, meaning that we have to split the three of them
        # in two at split = 2/3 = 0.66 = 66% (train on the first 2 and test on the
        # following)
        split = float(i-1)/i
        
        # example with i = 4 (first 4 folds):
        #      Splitting the first       4        chunks at          3      /        4
        print ('Splitting the first ' + str(i) + ' chunks at ' + str(i-1) + '/' + str(i) )
        
        # as we loop over the folds X and y are updated and increase in size.
        # This is the data that is going to be split and it increases in size 
        # in the loop as we account for more folds. If k = 300, with i starting from 2
        # the result is the following in the loop
        # i = 2
        # X = X_train[:(600)]
        # y = y_train[:(600)]
        #
        # i = 3
        # X = X_train[:(900)]
        # y = y_train[:(900)]
        # .... 
        X = X_train[:(k*i)]
        y = y_train[:(k*i)]
        print ('Size of train + test: ', X.shape) # the size of the dataframe is going to be k*i
 
        # X and y contain both the folds to train and the fold to test.
        # index is the integer telling us where to split, according to the
        # split percentage we have set above
        index = int(np.floor(X.shape[0] * split))
        
        # folds used to train the model        
        X_trainFolds = X[:index]        
        y_trainFolds = y[:index]
        
        # fold used to test the model
        X_testFold = X[(index + 1):]
        y_testFold = y[(index + 1):]
        
        #Classify Test Folds
        ts_knn = ts_KnnClassifier()
        print ('')
        print ('KNN Classifier Is Classifying Data Points')
        ts_knn.predict(X_trainFolds, X_testFold, y_trainFolds, w = 10)
        predictions = ts_knn.get_preds()
        y_predsCV.extend(predictions)
        
        #
        
        # i starts from 2 so the zeroth element in accuracies array is i-2. performClassification()
        #is a function which takes care of a classification problem. This is only an example and 
        #you can replace this function with whatever ML approach you need.
        f1_scores[i-2] = f1_score(y_testFold, predictions, average='weighted')
        precision_scores[i-2] =  precision_score(y_testFold, predictions, average='weighted')
        recall_scores[i-2] = recall_score(y_testFold, predictions, average='weighted')
        
        
#         # example with i = 4:
#         #      Accuracy on fold         4     :    0.85423
        print ('F1 Score on fold ' + str(i) + ': ', f1_scores[i-2])
        print ('Precision Score on fold ' + str(i) + ': ', precision_scores[i-2])
        print ('Recall Score on fold ' + str(i) + ': ', recall_scores[i-2])
    
    # the function returns the means of the precision_scores, recall scores, and f1_scores on the n-1 folds    
    return precision_scores.mean(), recall_scores.mean(), f1_scores.mean()

In [37]:
X_trainFolds, y_trainFolds, X_testFold, y_testFold = performTimeSeriesCV(matrix, labels, 10)

Size train set:  (941, 30)
Size of each fold:  94

Splitting the first 2 chunks at 1/2
Size of train + test:  (188, 30)

KNN Classifier Is Classifying Data Points


IndexError: ignored

In [48]:
ts_knn = ts_KnnClassifier()
ts_knn.predict(X_trainFolds, X_testFold, y_trainFolds, w = 10)
y_pred = ts_knn.get_preds()

TypeError: ignored

In [0]:
d1 = Counter(y_pred)
print d1

In [53]:
d2 = Counter(y_testFold)
print (d2)

NameError: ignored

In [51]:
print (ts_knn.performance(y_testFold))

NameError: ignored